In [1]:
%load_ext autoreload
%autoreload 2

%connect_info

{
  "shell_port": 41569,
  "iopub_port": 58909,
  "stdin_port": 53585,
  "control_port": 46457,
  "hb_port": 35561,
  "ip": "127.0.0.1",
  "key": "5141fa6b-fe25bbb6b5db0401f4ddb2c0",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-b443610d-489d-4192-92d0-bd65dc431860.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [2]:
import django
from django.db import transaction

django.setup()

In [3]:
CONFIG_SHEET_NAME = "CCBP Configuration_v4"

### Populate project and project roles

In [4]:
@transaction.atomic()
def populate_project_and_roles():
    from ib_tasks.populate.population_utils import populate_project_roles
    populate_project_roles(CONFIG_SHEET_NAME)
    

@transaction.atomic()
def populate_task_and_boards_related_data():
    
    configuration_sheets = [
        CONFIG_SHEET_NAME
    ]

    for configuration_sheet in configuration_sheets:

        from ib_tasks.populate.population_utils import populate_data
        populate_data(configuration_sheet)
        
        from ib_boards.populate.get_data_from_sheet_for_boards_and_columns import GetBoardsAndColumnsDataFromSheet
        loader = GetBoardsAndColumnsDataFromSheet()
        loader.get_data_from_sheet(configuration_sheet)
        loader.create_boards_and_columns(configuration_sheet)

        
@transaction.atomic()
def populate_project_related_data():

    from ib_tasks.populate.population_utils import populate_projects_for_task_templates
    populate_projects_for_task_templates(CONFIG_SHEET_NAME)
    
    from ib_boards.populate.get_data_from_sheet_for_project_boards import GetSheetDataForProjectBoards
    loader = GetSheetDataForProjectBoards()
    loader.get_data_from_project_boards_sub_sheet(CONFIG_SHEET_NAME)

@transaction.atomic()
def create_indices_in_elasticsearch():
    from elasticsearch_dsl import connections
    from django.conf import settings
    es = connections.create_connection(
        hosts=[settings.ELASTICSEARCH_ENDPOINT], timeout=20
    )

    from ib_tasks.documents.elastic_task import TASK_INDEX_NAME
    es.indices.create(index=TASK_INDEX_NAME)



    
    
@transaction.atomic()
def load_task_template_from_sheet():
#     create_indices_in_elasticsearch()÷
        
    populate_project_and_roles()
    populate_task_and_boards_related_data()
    populate_project_related_data()


In [5]:
load_task_template_from_sheet()

[BoardDTO(board_id='CCBPB_LEADS', name='India 4.0 Beneficiaries Calls'), BoardDTO(board_id='CCBPB_LEADS_STATUS', name='India 4.0 Beneficiaries Status')] [ColumnDTO(column_id='CCBPC_DRAFTS', name='Beneficiaries Drafts', display_order=1, task_template_stages=[TaskTemplateStagesDTO(task_template_id='CCBP_LGA', stages=['CCBP_DRAFTS'])], user_role_ids=['ALL_ROLES'], column_summary='ColumnSummary1', column_actions='ColumnSummary3', list_view_fields=[TaskSummaryFieldsDTO(task_id='ravi', summary_fields='ravi')], kanban_view_fields=[TaskSummaryFieldsDTO(task_id='ravi', summary_fields='ravi')], board_id='CCBPB_LEADS'), ColumnDTO(column_id='CCBP_LEADS', name='Beneficiaries', display_order=2, task_template_stages=[TaskTemplateStagesDTO(task_template_id='CCBP_LGA', stages=['CCBP_LEADS'])], user_role_ids=['ALL_ROLES'], column_summary='ColumnSummary1', column_actions='ColumnSummary3', list_view_fields=[TaskSummaryFieldsDTO(task_id='ravi', summary_fields='ravi')], kanban_view_fields=[TaskSummaryFields